In [1]:
# imports
import pandas as pd
import torch
from chess import pgn
import io
import chess
from tqdm.notebook import tqdm

In [2]:
path = 'C://Users//Anu//Downloads//archive//GM_games_dataset.csv'

In [3]:
df = pd.read_csv(path, nrows=80000, usecols=['pgn'])


In [4]:
chess.PIECE_TYPES

[1, 2, 3, 4, 5, 6]

In [5]:
# d = [king, queen, rook, bishop, knight, pawn, black, white, can_move]

def pgn_to_boardrep(game_pgn):
    game = pgn.read_game(game_pgn)
    moves = game.mainline_moves()
    board = chess.Board()
    pieces = chess.PIECE_TYPES
    states_arr = []
    moves_arr = []
    
    for move in moves:
        squares_arr = []
        for square in range(64):
            square_arr = ([board.piece_type_at(square) if board.piece_type_at(square) else 0] + [int(board.turn == board.color_at(square))])
            squares_arr.append(square_arr)
            
        
        moves_arr.append([move.from_square, move.to_square])
        
        
        board.push(move)
        
        states_arr.append(squares_arr)
        
    return (torch.tensor(states_arr, device='cpu'), torch.tensor(moves_arr, device='cpu'))

In [6]:
features = []
labels = []
games = tqdm(df['pgn'].items(), total=len(df))
for game_pgn in games:
    features.append(pgn_to_boardrep(io.StringIO(game_pgn[1]))[0])
    labels.append(pgn_to_boardrep(io.StringIO(game_pgn[1]))[1])


  0%|          | 0/80000 [00:00<?, ?it/s]

In [7]:
features = torch.vstack(features)
labels = torch.vstack(labels)

In [8]:
features

tensor([[[4, 1],
         [2, 1],
         [3, 1],
         ...,
         [3, 0],
         [2, 0],
         [4, 0]],

        [[4, 0],
         [2, 0],
         [3, 0],
         ...,
         [3, 1],
         [2, 1],
         [4, 1]],

        [[4, 1],
         [2, 1],
         [3, 1],
         ...,
         [3, 0],
         [2, 0],
         [4, 0]],

        ...,

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0],
         [0, 0],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]]])

In [9]:
torch.save({"features": features, "labels": labels}, 'data/dataset_v2.pt')

In [10]:
features.shape

torch.Size([6656432, 64, 2])